1) Using stock ticker being able to pull the available tickers for the corresponding FnO.
2) Being able to get dynamic symbols for the past 30 days to backtest strategies. First for a single stock and then for multiple stocks.

1) For expiry analysis, I should be able to figure out the end of day price range or the movement of price of the underlying using the trade data such as options, volumes of futures traded, option chain and corresponding changes in the option chain data 

In [3]:
import sqlite3
import pandas as pd
import os, sys
from datetime import datetime
from pandas.tseries.offsets import BDay

sys.path.append(os.path.dirname(os.getcwd()))
print(os.getcwd())
from utils.historical_data import hist_df, hist_data

db_path = os.path.join(os.path.dirname(os.getcwd()), "utils", "symbol_mappings.db")
print(db_path)
conn = sqlite3.connect(db_path)

# Read all data
df = pd.read_sql_query("SELECT * FROM symbol_mappings WHERE is_active = 1", conn)

# Display the data
print(f"Total records: {len(df)}")
print("\nData:")
print(df[['company_name', 'symbol', 'sector', 'market_cap', 'pe_ratio', 'lot_size']].to_string(index=False))

# Close connection
conn.close()

C:\Users\akshatx.agrawal\Strategies\expiry_analysis
C:\Users\akshatx.agrawal\Strategies\utils\symbol_mappings.db
Total records: 50

Data:
                   company_name     symbol         sector  market_cap  pe_ratio  lot_size
            Reliance Industries   RELIANCE      Oil & Gas   1871541.0     25.26     500.0
                      HDFC Bank   HDFCBANK        Banking   1458513.9     20.69     550.0
                  Bharti Airtel BHARTIARTL        Telecom   1101133.6     37.91     475.0
      Tata Consultancy Services        TCS             IT   1098234.3     22.29     175.0
                     ICICI Bank  ICICIBANK        Banking    986560.0     18.66     700.0
            State Bank of India       SBIN        Banking    773050.6     10.06     750.0
                 Bajaj Finance  BAJFINANCE           None    639916.2     36.78       NaN
                        Infosys       INFY             IT    620897.6     22.77     400.0
             Hindustan Unilever HINDUNILVR          

In [29]:
prev_day = 1
today = pd.date_range(start=datetime.today(), end = datetime.today())
yes = (today-BDay(prev_day)).strftime('%d/%m/%Y')[0] 

def yes_close(symbol):
    sym = f"NSE:{symbol}-EQ"
    data_for_day = hist_df(sym, yes, "1D")  # 5-minute candles for the specified day
    return data_for_day['Close'][0]

In [54]:
import json
import numpy as np
import os
import sys

def get_option_symbols_in_range(stock_symbol, underlying_price, percentile_range=5, expiry=None):
    """
    Extract CE and PE option symbols within ±percentile_range of underlying price
    
    Args:
        stock_symbol: Stock symbol (e.g., 'RELIANCE', 'TCS')
        underlying_price: Current underlying price
        percentile_range: Percentage range (default 5%)
        expiry: Optional expiry code (e.g., '25SEP'); if None, picks front/nearest
    
    Returns:
        dict: {'ce_symbols': [...], 'pe_symbols': [...], 'expiry': 'XXMMM', 'future_symbol': 'NSE:XXX...FUT'}
    """
    
    # Load options data
    with open(r'C:\Users\akshatx.agrawal\Strategies\utils\options_details_cache.json', 'r') as f:
        options_data = json.load(f)
    
    if stock_symbol not in options_data:
        return {'ce_symbols': [], 'pe_symbols': []}
    
    stock_data = options_data[stock_symbol]
    strikes = [float(s) for s in stock_data['strikes']]
    expiries = stock_data.get('expiries', [])
    if not expiries:
        return {'ce_symbols': [], 'pe_symbols': []}
    
    # Choose a single expiry: if provided and valid use it; otherwise assume the last entry is front/nearest
    chosen_expiry = expiry if (expiry and expiry in expiries) else expiries[-1]
    
    # Choose nearest future: try to match chosen_expiry; fallback to first available
    futures = stock_data.get('futures', [])
    future_symbol = None
    if futures:
        matching_futures = [f for f in futures if chosen_expiry in f]
        future_symbol = matching_futures[0] if matching_futures else futures[0]
    
    # Calculate price range
    lower_bound = underlying_price * (1 - percentile_range/100)
    upper_bound = underlying_price * (1 + percentile_range/100)
    
    # Find strikes in range
    strikes_in_range = [s for s in strikes if lower_bound <= s <= upper_bound]
    
    # Get CE and PE symbols for strikes in range
    ce_symbols = []
    pe_symbols = []
    
    for strike in strikes_in_range:
        strike_str = str(int(strike))
        ce_symbol = f"NSE:{stock_symbol}{chosen_expiry}{strike_str}CE"
        pe_symbol = f"NSE:{stock_symbol}{chosen_expiry}{strike_str}PE"
        
        if ce_symbol in stock_data['ce_symbols']:
            ce_symbols.append(ce_symbol)
        if pe_symbol in stock_data['pe_symbols']:
            pe_symbols.append(pe_symbol)
    
    return {
        'ce_symbols': ce_symbols,
        'pe_symbols': pe_symbols,
        'strikes_in_range': strikes_in_range,
        'price_range': (lower_bound, upper_bound),
        'expiry': chosen_expiry,
        'future_symbol': future_symbol
    }

In [55]:
for i in df['symbol'][:1]:
    underlying_price = yes_close(i)
    print(underlying_price)
    result = get_option_symbols_in_range(i, underlying_price, percentile_range=5)
    print(result)

ok
1363.4
{'ce_symbols': ['NSE:RELIANCE25SEP1300CE', 'NSE:RELIANCE25SEP1310CE', 'NSE:RELIANCE25SEP1320CE', 'NSE:RELIANCE25SEP1330CE', 'NSE:RELIANCE25SEP1340CE', 'NSE:RELIANCE25SEP1350CE', 'NSE:RELIANCE25SEP1360CE', 'NSE:RELIANCE25SEP1370CE', 'NSE:RELIANCE25SEP1380CE', 'NSE:RELIANCE25SEP1390CE', 'NSE:RELIANCE25SEP1400CE', 'NSE:RELIANCE25SEP1410CE', 'NSE:RELIANCE25SEP1420CE', 'NSE:RELIANCE25SEP1430CE'], 'pe_symbols': ['NSE:RELIANCE25SEP1300PE', 'NSE:RELIANCE25SEP1310PE', 'NSE:RELIANCE25SEP1320PE', 'NSE:RELIANCE25SEP1330PE', 'NSE:RELIANCE25SEP1340PE', 'NSE:RELIANCE25SEP1350PE', 'NSE:RELIANCE25SEP1360PE', 'NSE:RELIANCE25SEP1370PE', 'NSE:RELIANCE25SEP1380PE', 'NSE:RELIANCE25SEP1390PE', 'NSE:RELIANCE25SEP1400PE', 'NSE:RELIANCE25SEP1410PE', 'NSE:RELIANCE25SEP1420PE', 'NSE:RELIANCE25SEP1430PE'], 'strikes_in_range': [1300.0, 1310.0, 1320.0, 1330.0, 1340.0, 1350.0, 1360.0, 1370.0, 1380.0, 1390.0, 1400.0, 1410.0, 1420.0, 1430.0], 'price_range': (1295.23, 1431.5700000000002), 'expiry': '25SEP', 'f

In [44]:
os.getcwd()

'C:\\Users\\akshatx.agrawal\\Strategies\\expiry_analysis'

In [20]:
os.path.dirname(os.getcwd())

'C:\\Users\\akshatx.agrawal\\Strategies'

In [ ]:
!which python
!pip show fyers-apiv3

In [ ]:
Step 1: Data Collection & Engineering
    Minute-by-Minute Data Sources:
    Underlying prices (1-minute OHLCV)
    Options data (CE/PE prices, volumes, OI for multiple strikes)
    Futures data (prices, volumes, OI)
    Option chain changes (OI changes, volume spikes)
    Greeks (Delta, Gamma, Theta, Vega if available)
    Implied Volatility (IV changes)
    
    Feature Engineering:
    Volume ratios (options volume vs underlying volume)
    Put-Call ratios (by volume and OI)
    OI changes (delta OI for different strikes)
    Price momentum (rolling averages, RSI, MACD)
    Volatility measures (realized vs implied)
    Gamma exposure (estimated from OI)
    Time decay effects (theta exposure)

Step 2: Target Variable Creation
    Movement Classification:
    Direction: Up/Down/Neutral (with thresholds)
    Magnitude: Small (<0.3%), Medium (0.3-1%), Large (>1%)
    Time horizon: Next 5min, 15min, 30min, 1hr
    Confidence Scoring:
    Probability distribution over price ranges
    Expected move with confidence intervals

Step 3: Model Architecture Options
    Traditional ML:
    Random Forest (for feature importance)
    XGBoost/LightGBM (for non-linear patterns)
    SVM (for classification)
    Deep Learning:
    LSTM/GRU (for time series patterns)
    Transformer (for attention mechanisms)
    CNN-LSTM (for spatial-temporal patterns)
    Ensemble models (combining multiple approaches)

Step 4: Data Pipeline
    Real-time Processing:
    Streaming data ingestion
    Feature calculation on-the-fly
    Model inference every minute
    Confidence scoring and alerts
    Backtesting Framework:
    Walk-forward analysis
    Out-of-sample testing
    Performance metrics (accuracy, Sharpe ratio, max drawdown)

Step 5: Model Training Strategy
    Data Split:
    Training: 60% (historical data)
    Validation: 20% (for hyperparameter tuning)
    Testing: 20% (final evaluation)
    Cross-validation:
    Time series CV (no data leakage)
    Rolling window validation

Step 6: Performance Metrics
    Prediction Accuracy:
    Direction accuracy (up/down prediction)
    Magnitude accuracy (range prediction)
    Confidence calibration (reliability of confidence scores)
    Trading Metrics:
    Hit rate (profitable predictions)
    Risk-adjusted returns
    Maximum drawdown

Step 7: Implementation Phases
    Phase 1: Data Collection
    Set up minute-by-minute data feeds
    Build feature engineering pipeline
    Create target variables
    
    Phase 2: Model Development
    Start with simple models (Random Forest)
    Add complexity (LSTM, Transformer)
    Implement ensemble methods
    
    Phase 3: Backtesting
    Historical performance analysis
    Risk management rules
    Position sizing strategies
    
    Phase 4: Live Trading
    Real-time model deployment
    Monitoring and alerting
    Continuous model updates

Step 8: Key Considerations
    Market Regimes:
    Bull/Bear markets (different patterns)
    High/Low volatility periods
    Expiry effects (gamma squeeze, pinning)
    Risk Management:
    Position limits based on confidence
    Stop losses and take profits
    Portfolio diversification
    Model Maintenance:
    Regular retraining (weekly/monthly)
    Feature drift monitoring
    Performance degradation alerts
    This approach will give you a sophisticated system that can predict price movements with confidence intervals, perfect for options trading strategies!